<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/characterLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
import time

In [ ]:
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
text = open(data_dir,'rb').read().decode(encoding='utf8')

In [ ]:
vocab = sorted(set(text))
vocab_size = len(vocab)

In [ ]:
charToIdx = {u:i for i, u in enumerate(vocab)}
idxToChar = {i:u for i, u in enumerate(vocab)}
print(charToIdx)

In [ ]:
text_to_int = np.array([charToIdx[c] for c in text])
print(len(text_to_int))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [ ]:
char_dataset =tf.data.Dataset.from_tensor_slices(text_to_int)
sequences = char_dataset.batch(100, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(10000).batch(64, drop_remainder=True)
print(dataset)

In [ ]:

class RNN(tf.keras.Model):
 def __init__(self, batch_size):
   super(RNN, self).__init__()
   self.embedding_layer = tf.keras.layers.Embedding(vocab_size, 256,
                                                    batch_input_shape=[batch_size, None])
   self.hidden_layer_1 = tf.keras.layers.LSTM(1024,
                                             return_sequences=True,
                                             stateful=True,
                                             recurrent_initializer='glorot_uniform')
   self.output_layer = tf.keras.layers.Dense(vocab_size)

 def call(self, x):
   embedded_input = self.embedding_layer(x)
   features = self.hidden_layer_1(embedded_input)
   logits = self.output_layer(features)

   return logits

def sparse_cross_entropy_loss(labels, logits):
  return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels,
                                                                        logits, from_logits=True))
optimizer = tf.keras.optimizers.Adam()
@tf.function
def train_step(model, input, target): # backward(미분)
  with tf.GradientTape() as tape:
    logits = model(input)
    loss = sparse_cross_entropy_loss(target, logits)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
batch_size = 64
model = RNN(batch_size)
# model.summary()
epoch = 10

In [ ]:
for (batch_n, (input, target)) in enumerate(dataset):
    loss = train_step(model, input, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (epoch + 1) % 5 == 0:
        print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
        print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def generate_text(model, start_string):
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다.

    # start_sting을 integer 형태로 변환합니다.
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 샘플링 결과로 생성된 string을 저장할 배열을 초기화합니다.
    text_generated = []

    # 낮은 temperature 값은 더욱 정확한 텍스트를 생성합니다.
    # 높은 temperature 값은 더욱 다양한 텍스트를 생성합니다.
    temperature = 1.0

    # 여기서 batch size = 1 입니다.
    model.reset_states()
    for i in range(num_sampling):
        predictions = model(input_eval)
        # 불필요한 batch dimension을 삭제합니다.
        predictions = tf.squeeze(predictions, 0)

        # 모델의 예측결과에 기반해서 랜덤 샘플링을 하기위해 categorical distribution을 사용합니다.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 예측된 character를 다음 input으로 사용합니다.
        input_eval = tf.expand_dims([predicted_id], 0)
        # 샘플링 결과를 text_generated 배열에 추가합니다.
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print("샘플링을 시작합니다!")
batch_size = 1
sampling_model = RNN(batch_size)
print(generate_text(model, start_string=u' '))